<a href="https://www.kaggle.com/code/avikumart/customer-support-ticket-tagger?scriptVersionId=214535694" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
#!pip install transformers datasets torch scikit-learn

## Importing necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

In [ ]:
df = pd.read_csv("/kaggle/input/customer-support-ticket-tagging/customer_tickets.csv")
df.columns = ["text","labels"]
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels']) # converts labels which are in character to numerix format

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['text', 'labels']])
hf_dataset = dataset.train_test_split(test_size=0.145)
print(hf_dataset)

In [ ]:
label_encoder.classes_

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "microsoft/deberta-v3-base"  # loading the deberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_datasets = hf_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=0.00002,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=30,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("./text-classification-model")
tokenizer.save_pretrained("./text-classification-model")

In [ ]:
from transformers import pipeline
# loading the locally saved model
classifier = pipeline("text-classification", model="./text-classification-model", tokenizer=tokenizer)

In [ ]:
# Predictor Function to evaluate some tickets
def predictor(input_ticket,org_label):
    print(f"Input Ticket: {input_ticket}")
    result = classifier(input_ticket)
    print("\n")
    org_label_decoded = label_encoder.inverse_transform([int(org_label)])[0]
    decoded_label = label_encoder.inverse_transform([int(result[0]['label'].split("_")[-1])])[0]
    print("Original Label: ",org_label,",Original Label Decoded: ",org_label_decoded)
    print(f"Predicted Label: {int(result[0]['label'].split('_')[-1])} ,Predicted label Decoded: {decoded_label}")

In [ ]:
predictor(df['text'][319],df['labels'][319])

In [ ]:
predictor(df['text'][338],df['labels'][338])

In [ ]:
predictor(df['text'][317],df['labels'][317])

In [ ]:
predictor(df['text'][210],df['labels'][210])

In [ ]:
predictor(df['text'][50],df['labels'][50])

In [ ]:
predictor(df['text'][175],df['labels'][175])

In [ ]:
predictor(df['text'][15],df['labels'][15])